In [ ]:
import os
from os import listdir
from os.path import isfile, join

import re

import numpy as np

from datetime import datetime

import json

import pandas as pd

from IPython.display import display, Markdown

import uuid

import html

In [ ]:
import httpimport

with httpimport.remote_repo(['visualizador_dataframes','ferramentas_linguagem_natural'],'https://bonafe.github.io/CienciaDadosPython/src/componentes/'):
    from visualizador_dataframes import VisualizadorDataFrames
    from ferramentas_linguagem_natural import FerramentasLinguagemNatural

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)

In [ ]:
def prefixoData(data):
    return data.strftime("%Y_%m_%d")

In [ ]:
pasta_tcc_bonafe = os.path.dirname("z:/bases_capacita/projeto_capacita/tcc_bonafe/")
todos_arquivos_pasta_tcc_bonafe = listdir(pasta_tcc_bonafe)

print ('\n*************** Arquivos Base TCC Bonafé ***************\n\n')
print (*todos_arquivos_pasta_tcc_bonafe, sep = '\n')

In [ ]:

df_instrutores = pd.read_csv( os.path.join(pasta_tcc_bonafe,"2022_06_30-instrutores_acoes-capacitacao_conteudo-programatico.csv"), index_col=None, header=0)
df_instrutores.name = "instrutores"


# Anonimização dos dados pessoais

In [ ]:
def anonimizarSerie(df, coluna, coluna_anonimizada):
    
    df_copia = df.copy()
    
    dicionario = {}
    
    for indice, valor in df[coluna].iteritems():
        if valor not in dicionario:
            dicionario[valor] = str(uuid.uuid1())
        
    df_copia[coluna_anonimizada] = df_copia.apply (lambda linha: dicionario[linha[coluna]], axis=1)
    
    return df_copia
    

In [ ]:
df_instrutores = df_instrutores[df_instrutores['matricula_siape'].notna()]

In [ ]:
ds_anonimizado = anonimizarSerie(df_instrutores, "matricula_siape", "matricula_siape_anonimizada")
ds_anonimizado = anonimizarSerie(ds_anonimizado, "nome_instrutor", "nome_instrutor_anonimizado")
ds_anonimizado = anonimizarSerie(ds_anonimizado, "nome_rcd", "nome_rcd_anonimizado")

# Remoção dos nomes de pessoas das variáveis textos

In [ ]:
FerramentasLinguagemNatural.gerarDicionario(ds_anonimizado['nome_instrutor']).sort_values("contagem", ascending=False).head(10)

In [ ]:
FerramentasLinguagemNatural.gerarDicionario(ds_anonimizado['nome_rcd']).sort_values("contagem", ascending=False).head(10)

In [ ]:
dicionario_nomes = pd.concat([
    FerramentasLinguagemNatural.gerarDicionario(ds_anonimizado['nome_instrutor']),
    FerramentasLinguagemNatural.gerarDicionario(ds_anonimizado['nome_rcd'])
])

In [ ]:
lista_nomes = dicionario_nomes['palavra'].to_list()

In [ ]:
#Retira de uma string qualquer nome que apareça na lista de nomes
def retirarNomesString(string):
    
    string = str(string)
    
    #Apenas concatena palavras que não estão na lista de nomes (em minúsculo)
    return ' '.join([palavra for palavra in string.split() if palavra.lower() not in (lista_nomes)])

In [ ]:
#Remove todos as palavras que são nomes da coluna conteudo_programatico
ds_anonimizado['conteudo_programatico'] = ds_anonimizado['conteudo_programatico'].apply(retirarNomesString)

In [ ]:
#Remove todos as palavras que são nomes da coluna conteudo_programatico
ds_anonimizado['titulo_acao'] = ds_anonimizado['titulo_acao'].apply(retirarNomesString)

In [ ]:
#Remove todos as palavras que são nomes da coluna nome_curso
ds_anonimizado['nome_curso'] = ds_anonimizado['nome_curso'].apply(retirarNomesString)

In [ ]:
#Remove todos as palavras que são nomes da coluna objetivo_projeto
ds_anonimizado['objetivo_projeto'] = ds_anonimizado['objetivo_projeto'].apply(retirarNomesString)

# Remoção de caracteres especiais HTML das variáveis texto

In [ ]:

def removerCaracteresEspeciais(texto):
    texto = str(texto)
    return html.unescape(texto)

In [ ]:
ds_anonimizado['conteudo_programatico'] = ds_anonimizado['conteudo_programatico'].apply(removerCaracteresEspeciais)

In [ ]:
ds_anonimizado['titulo_acao'] = ds_anonimizado['titulo_acao'].apply(removerCaracteresEspeciais)

In [ ]:
ds_anonimizado['nome_curso'] = ds_anonimizado['nome_curso'].apply(removerCaracteresEspeciais)

In [ ]:
ds_anonimizado['objetivo_projeto'] = ds_anonimizado['objetivo_projeto'].apply(removerCaracteresEspeciais)

# Remoção das colunas com identificações pessoais

In [ ]:
ds_anonimizado.drop (columns=['matricula_siape', 'nome_instrutor', 'nome_rcd'], inplace=True)

In [ ]:
ds_anonimizado.info()

In [ ]:
pasta_destino = f'z:/bases_capacita/projeto_capacita/tcc_bonafe/'
nome_arquivo = f'{pasta_destino}/{prefixoData(datetime.now())}-instrutores_acoes-capacitacao_conteudo-programatico_anonimizado.csv'

In [ ]:
ds_anonimizado.to_csv (nome_arquivo, index=False)